- v4: better folding
    * supports ("1234567", "2345617", "3456271", "4563712", "5647123", "6571234", "6712345") -> "123456*712345"
    * but no improvement :(
- v3: Apply the [permutation rebalancing](https://www.kaggle.com/kostyaatarik/permutations-rebalancing/notebook) before optimization (2503 -> 2501)

In [ ]:
import itertools

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

In [ ]:
perms = list(map(lambda p: "".join(p), itertools.permutations("1234567")))
perm2id = {p: i for i, p in enumerate(perms)}

perms_arr = np.array([list(map(int, p)) for p in perms])
perms_arr.shape

## Create Cost matrix

In [ ]:
perms_onehot = np.eye(7)[perms_arr-1, :].transpose(0, 2, 1)
assert np.allclose(perms_onehot[:,0,:].astype(np.int64), (perms_arr == 1).astype(np.int64))

print("onehot 1234567:")
print(perms_onehot[perm2id["1234567"]])

print("onehot 5671234:")
print(perms_onehot[perm2id["5671234"]])

print("correlate between 1234567 and 5671234")
left = perms_onehot[perm2id["1234567"]]
right = perms_onehot[perm2id["5671234"]]
matches = F.conv2d(
    F.pad(torch.Tensor(left[None, None, :, :]), (7, 7)),
    torch.Tensor(right[None, None, :, :]),
    padding="valid"
).numpy().reshape(-1)
print(matches)
must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
must_match_right2left = np.array([0, 1, 2, 3, 4, 5, 6, 7, -1, -1, -1, -1, -1, -1, -1])
cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])
print("cost of 1234567 -> 5671234:", min(cost_ifmatch[np.equal(must_match_left2right, matches)]))
print("cost of 5671234 -> 1234567:", min(cost_ifmatch[np.equal(must_match_right2left, matches)]))

In [ ]:
M = F.conv2d(
    F.pad(torch.Tensor(perms_onehot[:, None, :, :]), (7, 7)),
    torch.Tensor(perms_onehot[:, None, :, :]),
    padding="valid"
).squeeze().numpy()

M.shape

In [ ]:
must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
must_match_left2right_wild = np.array([-1, -1, -1, -1, -1, -1, -1, 6, 5, 4, 3, 2, 1, 0, 0])

cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])

costMat = np.where(M == must_match_left2right, cost_ifmatch, np.inf).min(axis=-1).astype(np.int8)
costMatWild = np.minimum(costMat, np.where(M == must_match_left2right_wild, cost_ifmatch, np.inf).min(axis=-1)).astype(np.int8)

## Optim

In [ ]:
symbols = "🎅🤶🦌🧝🎄🎁🎀"
schedule = pd.read_csv("../input/santa-2021-tsp-baseline-2500/submission.csv").schedule.tolist()
words = [s.translate(str.maketrans(symbols, "1234567")) for s in schedule]

list(map(len, words))

In [ ]:
"""
Permutations Rebalancing (https://www.kaggle.com/kostyaatarik/permutations-rebalancing/notebook)

This leads a relaxation of constraints and sometimes you can find a better solution of wildcard positions.
"""

def find_strings_perms(strings, verbose=False):
    global perms
    found_perms = []
    for s in strings:
        found_perms.append([])
        for i in range(len(s)-6):
            p = s[i:i+7]
            if p in perms:
                found_perms[-1].append(p)
    if verbose:
        lens = [len(_) for _ in  found_perms]
        print(f'There are {lens} permutations in strings, {sum(lens)} in total.')
        lens = [len(set(_)) for _ in  found_perms]
        print(f'There are {lens} unique permutations in strings, {sum(lens)} in total.')
    return found_perms

def rebalance_perms(strings_perms, verbose=False):
    # convert to dicts for fast lookup and to keep permutations order
    strings_perms = [dict.fromkeys(_) for _ in strings_perms] 
    for p in strings_perms[0].copy():  # iterate over the copy to allow modification during iteration
        if p[:2] != "12" and (p in strings_perms[1] or p in strings_perms[2]):
            strings_perms[0].pop(p)
    for p in strings_perms[1].copy():
        if p[:2] != "12" and p in strings_perms[2]:
            strings_perms[1].pop(p)
    if verbose:
        lens = [len(_) for _ in  strings_perms]
        print(f'There are {lens} permutations left in strings after rebalancing, {sum(lens)} in total.')
    return [list(_) for _ in strings_perms] 

found_perms = find_strings_perms(words, verbose=True)
balanced_perms = rebalance_perms(found_perms, verbose=True)

In [ ]:
def wildcard(input_perms):
    new_words = []
    
    for i in range(len(input_perms)):
        nodes = [perm2id[p] for p in input_perms[i]]

        """
        0: no wild
        1-14: # of digits appeared after the first wild
        15-22: (# of digits appeared after the second wild) + 14
        """
        table = np.full((len(nodes), 23), 1e7, dtype=np.int64)
        backtrack = np.zeros((len(nodes), 23), dtype=np.int64)
        table[0, :] = 7
        for i in range(1, len(nodes)):
            e = costMat[nodes[i-1], nodes[i]]
            ew = costMatWild[nodes[i-1], nodes[i]]

            # non-wild transition
            table[i,0] = table[i-1,0] + e
            backtrack[i,0] = 0
            for j in range(1, 15):
                if table[i, min(14, j + e)] > table[i-1, j] + e:
                    table[i, min(14, j + e)] = table[i-1, j] + e
                    backtrack[i, min(14, j + e)] = j
            for j in range(15, 23):
                if table[i, min(22, j + e)] > table[i-1, j] + e:
                    table[i, min(22, j + e)] = table[i-1, j] + e
                    backtrack[i, min(22, j + e)] = j

            if ew < e:
                """
                i-1:  00000*0
                i  :    000*000
                             == ew
                        ===     mis
                            === distance_from_wild
                """
                left = np.array(list(map(int, perms[nodes[i-1]][ew:])))
                right = np.array(list(map(int, perms[nodes[i]][:-ew])))
                mis = np.where(left != right)[0][0]
                distance_from_wild = 7 - mis - 1

                # first wild transition
                if table[i, distance_from_wild] >= table[i-1, 0] + ew:
                    table[i, 1] = table[i-1, 0] + ew
                    backtrack[i, 1] = 0

                # second wild transition
                ## position of the second wild card is same as the previous one
                ## <=> table[i-1, distance_from_wild-ew]->table[i, distance_from_wild]
                if distance_from_wild - ew >= 1 and table[i, distance_from_wild] >= table[i-1, distance_from_wild-ew] + ew:
                    table[i, distance_from_wild] = table[i-1, distance_from_wild-ew] + ew
                    backtrack[i, distance_from_wild] = distance_from_wild-ew
                ## position of the third wild card is same as the previous one
                if distance_from_wild - ew >= 1 and table[i, distance_from_wild+14] >= table[i-1, distance_from_wild-ew+14] + ew:
                    table[i, distance_from_wild+14] = table[i-1, distance_from_wild-ew+14] + ew
                    backtrack[i, distance_from_wild+14] = distance_from_wild-ew+14

                ## put the second wild card and it is >=7 digits far from the previous one
                for j in range(1, 15):
                    if j + ew - distance_from_wild >= 7:
                        if table[i, distance_from_wild+14] >= table[i-1, j] + ew:
                            table[i, distance_from_wild+14] = table[i-1, j] + ew
                            backtrack[i, distance_from_wild+14] = j
        print(table[-1].min(), table[-1])

        # backtrack
        ns = [perms[nodes[-1]]]
        track = np.argmin(table[-1])
        wild = []
        for i in range(len(nodes)-2, -1, -1):
            e = costMat[nodes[i], nodes[i+1]]
            ew = costMatWild[nodes[i], nodes[i+1]]
            next_track = backtrack[i+1, track]
            if table[i+1, track] == table[i, next_track] + e:
                ns.append(perms[nodes[i]][:e])
            else:
                left = np.array(list(map(int, perms[nodes[i]][ew:])))
                right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                mis = np.where(left != right)[0][0]
                wild.append(table[i, next_track]-7+ew+mis)
                ns.append(perms[nodes[i]][:ew])
            track = next_track
        assert track == 0
        nsw = list("".join(ns[::-1]))
        for w in wild:
            nsw[w] = "*"
        new_words.append("".join(nsw))
    return new_words

new_words = wildcard(balanced_perms)

In [ ]:
print("score: ", max(map(len, words)), "->", max(map(len, new_words)))

In [ ]:
submission = pd.Series([a.translate(str.maketrans("1234567*", symbols+"🌟")) for a in new_words], name='schedule')
submission.to_csv('submission.csv', index=False)